<font size=50 color=darkblue>Max Flow LP</font>

# Problem modelling with DOcplex

## Import necessary modules

- DOcplex will be used to model and solve the Max-Flow LP
- Import module `itertools` to group links based on their tails/heads

In [ ]:
from docplex.mp.model import Model
import itertools

## Max-Flow LP <font size=3>(variables are colored blue)</font>
**Maximize**
### $${\color{blue}y}_{({\color{red}t},{\color{green}s})}$$
**Subject to**
### \begin{align*}
\sum_{(u,v)\in{\color{red}\delta}^{{\color{red}-}}_{(v)}}{\color{blue}y_{(u,v)}} - \sum_{(v,u)\in{\color{green}\delta}^{{\color{green}+}}_{(v)}}{\color{blue}y_{(v,u)}} = 0,\qquad&\forall v\in \mathcal{V}\\
0\le {\color{blue}y_{(u,v)}}\le\mu_{(u,v)},\qquad&\forall (u,v)\in \mathcal{E}
\end{align*}

## Display the network
<img src='img/mxf.png' width=700/>

## Define the model input
### \begin{align*}
\mathcal{V} =\,& \{0, 1, 2, 3, 4, 5, 6, 7\}&\\
\mathcal{E} =\,& \{(0,1), (0,2), (0,3), (1,2), (1,4), (1,5), (2,3), (2,5), (3,6), (4,5), (4,7), (5,6), (5,7), (6,2), (6,7)\}&\\
{\color{green}s} =\,& 0&\\
{\color{red}t} =\,& 7&
\end{align*}

In [ ]:
V = [0, 1, 2, 3, 4, 5, 6, 7]
E = [(0,1), (0,2), (0,3), (1,2), (1,4), (1,5), (2,3), (2,5), (3,6), (4,5), (4,7), (5,6), (5,7), (6,2), (6,7)]
s, t = 0, 7

## Augment the graph with the imaginary link (<font color='red'>t</font>, <font color='green'>s</font>)
### $$\mathcal{E}:=\mathcal{E}\cup \{({\color{red}t},{\color{green}s})\}$$

In [ ]:
E.append((t,s))

## Define in-links <code><font color='red'>delta_min</font></code> and out-links <code><font color='green'>delta_pls</font></code>, subscriptable by node indices
### \begin{align*}
{\color{red}\delta}^{\color{red}-}_{(v)} = \{e\in \mathcal{E}\,|&\,\text{head}(e)\equiv v \},&\quad\forall v\in \mathcal{V}\\
{\color{green}\delta}^{\color{green}+}_{(v)} = \{e\in \mathcal{E}\,|&\,\text{tail}(e)\equiv v \},&\quad\forall v\in \mathcal{V}\\
\end{align*}

In [ ]:
# Anonymous functions
head = lambda e: e[1]
tail = lambda e: e[0]
delta_min = {v: set(delt_min) for v, delt_min in itertools.groupby(sorted(E, key=head), head)}
delta_pls = {v: set(delt_pls) for v, delt_pls in itertools.groupby(sorted(E, key=tail), tail)}

## Define link capacities $\mu_{(e)},\,\forall e\in \mathcal{E}$, subscriptable by link indices
**<font color='red'>Note: The imaginary link</font> (<font color='red'>t</font>, <font color='green'>s</font>) <font color='red'> is unbounded from above. Its upper bound is set to <code>None</code></font>.**

In [ ]:
mu = dict(zip(E, [10, 5, 15, 4, 9, 15, 4, 8, 30, 15, 10, 15, 10, 6, 10, None]))

## Create the LP model for Max-Flow problem using DOcplex

In [ ]:
mxf_LP = Model(name='Max Flow')

## Define decision variables: $0\le {\color{blue}y_{(e)}}\le\mu_{(e)},\quad\forall e\in \mathcal{E}$

In [ ]:
mxf_LP.clear() # This line is optional. Its purpose is to avoid variable duplicates
y = {(u,v): mxf_LP.continuous_var(name=f'y({u},{v})', ub=mu[u,v]) for u,v in E}

## Define flow conservation constraints, subscriptable by node indices $$\sum_{(u,v)\in{\color{red}\delta}^{{\color{red}-}}_{(v)}}{\color{blue}y_{(u,v)}} - \sum_{(v,u)\in{\color{green}\delta}^{{\color{green}+}}_{(v)}}{\color{blue}y_{(v,u)}} = 0,\quad\forall v\in \mathcal{V}$$

In [ ]:
flow_conserv_ct = {v: mxf_LP.add_constraint(ct=mxf_LP.sum(y[e] for e in delta_min[v]) - mxf_LP.sum(y[e] for e in delta_pls[v]) == 0,
                                            ctname=f'flow_conservation({v})') for v in V}

## Define objective function $$\textbf{maximize}\qquad {\color{blue}y}_{({\color{red}t},{\color{green}s})}$$

In [ ]:
mxf_LP.maximize(y[t,s])

## Summarize the model

In [ ]:
mxf_LP.print_information()

## Solve the LP and display the result

In [ ]:
mxf_sol = mxf_LP.solve()
if mxf_sol:
    mxf_sol.display()

# Result Visualization

## Import visualization modules
- `igraph`, `matplotlib`

In [ ]:
import igraph as ig
import matplotlib.pyplot as plt

## For convenience, extract the solution to a dictionary named `sol_y`

In [ ]:
sol_y = mxf_sol.get_value_dict(y)
sol_y

## Since the link (<font color='red'>t</font>, <font color='green'>s</font>) is imaginary, remove it from the link list `E`

In [ ]:
E.remove((t,s))

## Instantiate a `Graph` object with module `igraph`
### Notes
- __*Node(s)*__ is/are called __*vertex/vertices*__ in `igraph`
- __*Link(s)*__ is/are called __*edge/edges*__ in `igraph`
- The edge list is sufficient to instantiate a `Graph` object. The vertex list is automatically inferred by `igraph` (based on the tails/heads' indices).

In [ ]:
g = ig.Graph(edges=E, directed=True)

## Visualize the graph

In [ ]:
g.vs['label'] = g.vs.indices
g.vs['size'] = 50
g.vs['color'] = 'white'
g.vs[s,t]['color'] = 'green', 'red'

lnk_flow_max = max(sol_y.values())
g.es['flow'] = [sol_y[e] for e in E]
g.es['cap'] = [mu[e] for e in E]
g.es['label'] = ' '
g.es['arrow_size'] = [10 if sol_y[e] > 0 else 6 for e in E]
g.es['arrow_width'] = [10 if sol_y[e] > 0 else None for e in E]
g.es['width'] = [.4*(sol_y[e] + 1) for e in E]
g.es['color'] = ['darkblue' if sol_y[e] > 0 else None for e in E]
g.es['flow_label_color'] = ['blue' if sol_y[e] > 0 else 'black' for e in E]
g.es['cap_label_color']  = ['red' if sol_y[e] > 0 else 'black' for e in E]
g.es['label_size'] = [10 if sol_y[e] > 0 else 7 for e in E]

fig, ax = plt.subplots()
fig.set_size_inches(10,10)

p = ig.plot(g, layout=g.layout('fr'), edge_label=g.es['label'], target=ax)

for e, edge_label in zip(g.es, p.get_edge_labels()):
    txt = ax.annotate(f'{e["flow"]:g}', xycoords=edge_label, xy=(-5,0), color=e['flow_label_color'], backgroundcolor='white', size=e['label_size'])
    txt =  ax.annotate(f' | ', xycoords=txt, xy=(1,0), color='k', size=e['label_size'], verticalalignment='bottom')
    txt =  ax.annotate(f'{e["cap"]}', xycoords=txt, xy=(1,0), color=e['cap_label_color'], backgroundcolor='white', size=e['label_size'], verticalalignment='bottom')
    
plt.show()